In [4]:
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import math
import sys

import skinny
import helpers 
import dpautils

from scipy import stats
from array import array
from operator import xor

np.set_printoptions(threshold=sys.maxsize)

In [5]:

TK1_0=[0,1,2,3] #Three times the traces
TK1_1=[4,5,6,7]
TK1_2_unsorted=[0,1,2,3] # Three times the traces
Tk1_3_unsorted=[8,9,10,11]

TK1=[]

Ns=list(range(1,10,1))
number_of_experiments=200

STD=[0.3,0.5,0.7]

In [6]:
def simultanous_attack(Ns,keys):
    success_rate_pr_n=[]
    for n in Ns:
        probs=[]
        for key in keys:
            P=helpers.gen_plaintexts(n)
            val=dpautils.compute_intemediate_values(P,int(key))
            interm_values=val[0]
            clear_text=val[1]
            TK1=val[2].A1


            T = dpautils.gen_traces(interm_values, std) # Only need to generate traces once.

            key_guess=[]

            for i in range(len(TK1_0)):
                kdi=helpers.determine_kdi(TK1_0[i]) 

                V = []
                scores=[]
                for ind in kdi:
                    V.append(np.matrix(dpautils.compute_v(ind,clear_text)))
                scores=dpautils.distinguisher_multivariate(V,T,kdi)
                key_guess.append(scores.index(max(scores)))

            #_____SUCCESS/FAIL_________________
            if(np.array_equal(TK1[0:4],key_guess)):
                probs.append(1)
            else:
                probs.append(0)
        success_rate_pr_n.append(np.average(probs))

    return success_rate_pr_n




def majority_vote_attack(Ns,keys):
    success_rate_pr_n=[]
    for n in Ns:
        probs=[]
        for key in keys:
            P=helpers.gen_plaintexts(n)
            val=dpautils.compute_intemediate_values(P,int(key))
            interm_values=val[0]
            clear_text=val[1]
            TK1=val[2].A1


            T = dpautils.gen_traces(interm_values, std) # Only need to generate traces once.

            key_guess_maj=[]

            for i in range(len(TK1_0)):
                kdi=helpers.determine_kdi(TK1_0[i]) 

                V = []
                scores=[]

                for ind in kdi:
                    V.append(np.matrix(dpautils.compute_v(ind,clear_text)))
                    scores.append(dpautils.distinguisher(V[kdi.index(ind)],T,ind))

                argmax_scores = [score.index(max(score)) for score in scores] #Argmax of list of scores
                key_guess_maj.append(dpautils.majority_vote_selector(argmax_scores))# Majority vote

            #_____SUCCESS/FAIL_________________
            if(np.array_equal(TK1[0:4],key_guess_maj)):
                probs.append(1)
            else:
                probs.append(0)
        success_rate_pr_n.append(np.average(probs))
    return success_rate_pr_n





def unanimous_attack(Ns,keys):
    success_rate_pr_n=[]
    for n in Ns:
        probs=[]
        for key in keys:
            P=helpers.gen_plaintexts(n)
            val=dpautils.compute_intemediate_values(P,int(key))
            interm_values=val[0]
            clear_text=val[1]
            TK1=val[2].A1


            T = dpautils.gen_traces(interm_values, std) # Only need to generate traces once.

            key_guess_unanimous=[]

            for i in range(len(TK1_0)):
                kdi=helpers.determine_kdi(TK1_0[i]) 

                V = []
                scores=[]

                for ind in kdi:
                    V.append(np.matrix(dpautils.compute_v(ind,clear_text)))
                    scores.append(dpautils.distinguisher(V[kdi.index(ind)],T,ind))

                argmax_scores = [score.index(max(score)) for score in scores] #Argmax of list of scores 

                key_guess_unanimous.append(dpautils.unanimous_selector(argmax_scores)) # Unanimous 

            #_____SUCCESS/FAIL_________________
            if(np.array_equal(TK1[0:4],key_guess_unanimous)):
                probs.append(1)
            else:
                probs.append(0)
        success_rate_pr_n.append(np.average(probs))
    return success_rate_pr_n



def individual_attack(Ns,keys):
    success_rate_pr_n=[]
    for n in Ns:
        probs=[]
        for key in keys:
            P=helpers.gen_plaintexts(n)
            val=dpautils.compute_intemediate_values(P,int(key))
            interm_values=val[0]
            clear_text=val[1]
            TK1=val[2].A1


            T = dpautils.gen_traces(interm_values, std) # Only need to generate traces once.

            key_guess=[]

            for i in range(len(TK1_0)):
                V = np.matrix(dpautils.compute_v(TK1_0[i],clear_text))
                scores= dpautils.distinguisher(V,T,TK1_0[i])
                key_guess.append(scores.index(max(scores)))
                
            #_____SUCCESS/FAIL_________________
            if(np.array_equal(TK1[0:4],key_guess)):
                probs.append(1)
            else:
                probs.append(0)
                
        success_rate_pr_n.append(np.average(probs))
    return success_rate_pr_n

def individual_attack_tk11(Ns,keys):
    success_rate_pr_n=[]
    for n in Ns:
        probs=[]
        for key in keys:
            P=helpers.gen_plaintexts(n)
            val=dpautils.compute_intemediate_values(P,int(key))
            interm_values=val[0]
            clear_text=val[1]
            TK1=val[2].A1


            T = dpautils.gen_traces(interm_values, std) # Only need to generate traces once.

            key_guess=[]

            for nibble in TK1_1:
                V = np.matrix(dpautils.compute_v(nibble,clear_text))
                scores= dpautils.distinguisher(V,T,nibble)
                key_guess.append(scores.index(max(scores)))
                
            #_____SUCCESS/FAIL_________________
            if(np.array_equal(TK1[0:4],key_guess)):
                probs.append(1)
            else:
                probs.append(0)
                
        success_rate_pr_n.append(np.average(probs))
    return success_rate_pr_n      

In [7]:
res=[]
for std in STD:
    res_per_std=[]
    keys=np.random.randint(2147483647, 9223372036854775807, size=number_of_experiments, dtype=np.int64)     
    res_per_std.append(simultanous_attack(Ns,keys))
    res_per_std.append(majority_vote_attack(Ns,keys))
    res_per_std.append(unanimous_attack(Ns,keys))
    res_per_std.append(individual_attack(Ns,keys))
    res.append(res_per_std)
    outfile="TK1-10-recovery-rate-std-"+str(std)+".png"

    helpers.plot_attacks(res_per_std[0], res_per_std[1], 
                 res_per_std[2], res_per_std[3],
                 "simultanous","majority vote",
                 "unanimous","individual",
                outfile)
    

KeyboardInterrupt: 

# TK1_1
For TK1_1 we only have a single nibble's leakage "per trace", so the best performing distinguisher here should be the individual attack.

In [ ]:
def individual_attack_tk11(Ns,keys):
    success_rate_pr_n=[]
    for n in Ns:
        probs=[]
        for key in keys:
            P=helpers.gen_plaintexts(n)
            val=dputils.compute_intemediate_values(P,int(key))
            interm_values=val[0]
            clear_text=val[1]
            TK1=val[2].A1


            T = dputils.gen_traces(interm_values, std) # Only need to generate traces once.

            key_guess=[]

            for nibble in TK1_1:
                V = np.matrix(dputils.compute_v(nibble,clear_text))
                scores= dputils.distinguisher(V,T,nibble)
                key_guess.append(scores.index(max(scores)))
                
            #_____SUCCESS/FAIL_________________
            if(np.array_equal(TK1[0:4],key_guess)):
                probs.append(1)
            else:
                probs.append(0)
                
        success_rate_pr_n.append(np.average(probs))
    return success_rate_pr_n      

In [ ]:
succes_rates_tk_11=[]
for std in STD:
    keys=np.random.randint(2147483647, 9223372036854775807, size=number_of_keys, dtype=np.int64)     
    succes_rates_tk_11.append(individual_attack_tk11(Ns,keys))
    outfile="TK1-11-recovery-rate-std-"+str(std)+".png"
    helpers.plot_single_attack(succes_rates_tk_11,"individual attack",outfile)

# Second half of TK1

Second half of retrieving TK1. Exact same as First half but with other values. 

In [ ]:
def simultanous_attack(Ns,keys):
    success_rate_pr_n=[]
    for n in Ns:
        probs=[]
        for key in keys:
            P=helpers.gen_plaintexts(n)
            val=dputils.compute_intemediate_values_round_2(P,int(key))
            interm_values=val[0]
            clear_text=val[1]
            TK1=val[2].A1


            T = dputils.gen_traces(interm_values, std) # Only need to generate traces once.

            key_guess=[]

            for i in range(len(TK1_0)):
                kdi=helpers.determine_kdi(TK1_0[i]) 

                V = []
                scores=[]
                for ind in kdi:
                    V.append(np.matrix(dputils.compute_v(ind,clear_text)))
                scores=dputils.distinguisher_multivariate(V,T,kdi)
                key_guess.append(scores.index(max(scores)))

            #_____SUCCESS/FAIL_________________
            if(np.array_equal(TK1[0:4],key_guess)):
                probs.append(1)
            else:
                probs.append(0)
        success_rate_pr_n.append(np.average(probs))

    return success_rate_pr_n

def majority_vote_attack(Ns,keys):
    success_rate_pr_n=[]
    for n in Ns:
        probs=[]
        for key in keys:
            P=helpers.gen_plaintexts(n)
            val=dputils.compute_intemediate_values_round_2(P,int(key))
            interm_values=val[0]
            clear_text=val[1]
            TK1=val[2].A1


            T = dputils.gen_traces(interm_values, std) # Only need to generate traces once.

            key_guess_maj=[]

            for i in range(len(TK1_0)):
                kdi=helpers.determine_kdi(TK1_0[i]) 

                V = []
                scores=[]

                for ind in kdi:
                    V.append(np.matrix(dputils.compute_v(ind,clear_text)))
                    scores.append(dputils.distinguisher(V[kdi.index(ind)],T,ind))

                argmax_scores = [score.index(max(score)) for score in scores] #Argmax of list of scores
                key_guess_maj.append(dputils.majority_vote_selector(argmax_scores))# Majority vote

            #_____SUCCESS/FAIL_________________
            if(np.array_equal(TK1[0:4],key_guess_maj)):
                probs.append(1)
            else:
                probs.append(0)
        success_rate_pr_n.append(np.average(probs))
    return success_rate_pr_n

def unanimous_attack(Ns,keys):
    success_rate_pr_n=[]
    for n in Ns:
        probs=[]
        for key in keys:
            P=helpers.gen_plaintexts(n)
            val=dputils.compute_intemediate_values_round_2(P,int(key))
            interm_values=val[0]
            clear_text=val[1]
            TK1=val[2].A1


            T = dputils.gen_traces(interm_values, std) # Only need to generate traces once.

            key_guess_unanimous=[]

            for i in range(len(TK1_0)):
                kdi=helpers.determine_kdi(TK1_0[i]) 

                V = []
                scores=[]

                for ind in kdi:
                    V.append(np.matrix(dputils.compute_v(ind,clear_text)))
                    scores.append(dputils.distinguisher(V[kdi.index(ind)],T,ind))

                argmax_scores = [score.index(max(score)) for score in scores] #Argmax of list of scores 

                key_guess_unanimous.append(dputils.unanimous_selector(argmax_scores)) # Unanimous 

            #_____SUCCESS/FAIL_________________
            if(np.array_equal(TK1[0:4],key_guess_unanimous)):
                probs.append(1)
            else:
                probs.append(0)
        success_rate_pr_n.append(np.average(probs))
    return success_rate_pr_n  

def individual_attack_tk11(Ns,keys):
    success_rate_pr_n=[]
    for n in Ns:
        probs=[]
        for key in keys:
            P=helpers.gen_plaintexts(n)
            val=dputils.compute_intemediate_values_round_2(P,int(key))
            interm_values=val[0]
            clear_text=val[1]
            TK1=val[2].A1


            T = dputils.gen_traces(interm_values, std) # Only need to generate traces once.

            key_guess=[]

            for nibble in TK1_1:
                V = np.matrix(dputils.compute_v(nibble,clear_text))
                scores= dputils.distinguisher(V,T,nibble)
                key_guess.append(scores.index(max(scores)))
                
            #_____SUCCESS/FAIL_________________
            if(np.array_equal(TK1[0:4],key_guess)):
                probs.append(1)
            else:
                probs.append(0)
                
        success_rate_pr_n.append(np.average(probs))
    return success_rate_pr_n      

In [ ]:
second_half_0=[]
for std in STD:
    res_per_std=[]
    keys=np.random.randint(2147483647, 9223372036854775807, size=number_of_experiments, dtype=np.int64)     
    res_per_std.append(simultanous_attack(Ns,keys))
    res_per_std.append(majority_vote_attack(Ns,keys))
    res_per_std.append(unanimous_attack(Ns,keys))
    res_per_std.append(individual_attack(Ns,keys))
    second_half.append(res_per_std)
    outfile="TK1-20-recovery-rate-std-"+str(std)+".png"

    helpers.plot_attacks(res_per_std[0], res_per_std[1], 
                 res_per_std[2], res_per_std[3],
                 "simultanous","majority vote",
                 "unanimous","individual",
                outfile)

In [ ]:
succes_rates_tk_11=[]
for std in STD:
    keys=np.random.randint(2147483647, 9223372036854775807, size=number_of_keys, dtype=np.int64)     
    succes_rates_tk_11.append(individual_attack_tk11(Ns,keys))
    outfile="TK1-21-recovery-rate-std-"+str(std)+".png"
    helpers.plot_single_attack(succes_rates_tk_11,"individual attack",outfile)